In [2]:
from nj_crashes.paths import PUBLIC_DIR, NJDOT_DIR
from nj_crashes.crashes import Crashes
from utz import *

import duckdb

In [3]:
df = Crashes.load()

In [4]:
df

,year,cc,mc,case,cn,mn,pdc,pdn,station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24


In [8]:
bkt = 'duckdb-wasm-test'
pre = f's3://{bkt}'
dir = path.join(env['c'], 'duckdb-wasm-test')

In [21]:
def make_parquet(k):
    print(k)
    n = int(float(k))
    d = df.iloc[:n]
    d.to_parquet(f'{pre}/{k}.parquet')    

In [28]:
keys = [ '1e5', '2e5', '5e5', '1e6', '2e6', '4e6', '6e6', ]
for k in keys:
    make_parquet(k)

1e5
2e5
4e5
8e5


In [30]:
make_parquet('5e5')

5e5


In [25]:
import boto3
s3 = boto3.client('s3')

In [26]:
def make_db(idx, index=False):
    out_name = f'{idx}-idx' if index else f'{idx}'
    db_path = f'{out_name}.duckdb'
    pqt_path = f'{idx}.parquet'
    with duckdb.connect(db_path) as con:
        con.sql(f"CREATE TABLE crashes AS SELECT * FROM '{pre}/{pqt_path}';")
        if index:
            con.sql(f"CREATE UNIQUE INDEX crashes_id_idx ON crashes (id);")
    with open(db_path, 'rb') as f:
        s3.put_object(Bucket=bkt, Key=db_path, Body=f)

In [31]:
def make_dbs(k):
    make_db(k)
    make_db(k, index=True)    

In [32]:
make_dbs('5e5')

In [29]:
for k in keys:
    print(k)
    make_dbs(k)

1e5
2e5
4e5


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

8e5


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
df1 = Crashes.load(2021)
df2 = Crashes.load(range(2019, 2022))
df3 = Crashes.load(range(2012, 2022))
len(df1), len(df2), len(df3)

(226958, 705630, 2681944)

In [4]:
df1.to_parquet(f'{pre}/1.parquet')

In [5]:
df2.to_parquet(f'{pre}/2.parquet')

In [24]:
df3.to_parquet(f'{pre}/3.parquet')

In [19]:
make_db(1)
make_db(1, index=True)
make_db(2)
make_db(2, index=True)

CatalogException: Catalog Error: Table with name "crashes" already exists!

In [25]:
make_db(3)
make_db(3, index=True)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
# db_path = 'crashes.duckdb'
db_path = '2021.duckdb'
con = duckdb.connect(db_path)
dsql = con.sql

In [4]:
def tables():
    return dsql("SHOW TABLES").df()

In [5]:
def indices():
    return dsql("SELECT * from duckdb_indexes()").df()

In [6]:
tbls = tables()
if not tbls[tbls.name == 'crashes'].empty:
    dsql("DROP TABLE crashes")
tables()

,name


In [7]:
indices()

,database_name,database_oid,schema_name,schema_oid,index_name,index_oid,table_name,table_oid,is_unique,is_primary,expressions,sql


In [8]:
dsql("VACUUM ANALYZE")

In [13]:
def db_size():
    return dsql("PRAGMA database_size").df()

In [11]:
con.close()

In [12]:
%%time
dsql(f"CREATE TABLE crashes AS SELECT * FROM '{pqt_path}';")

CPU times: user 713 ms, sys: 37.9 ms, total: 750 ms
Wall time: 787 ms


In [14]:
db_size()

,database_name,database_size,block_size,total_blocks,used_blocks,free_blocks,wal_size,memory_usage,memory_limit
0,2021,14.6MB,262144,56,54,2,0 bytes,14.1MB,13.7GB


In [15]:
%%time
dsql(f"CREATE UNIQUE INDEX crashes_id_idx ON crashes (id);")

CPU times: user 56.6 ms, sys: 14.4 ms, total: 71 ms
Wall time: 64.5 ms


In [16]:
dsql("CHECKPOINT")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [12]:
%%time
dsql(f"CREATE INDEX crashes_dt_icn_severity_idx ON crashes (Date, \"County Code\", severity);")

CPU times: user 201 ms, sys: 11.6 ms, total: 213 ms
Wall time: 87.7 ms


In [14]:
con.commit()

In [17]:
con.close()

In [28]:
# %%time
# dsql("EXPORT DATABASE 'crashes.duckdb'")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 14.1 s, sys: 4.19 s, total: 18.3 s
Wall time: 5.62 s


In [36]:
dsql("SELECT * FROM crashes WHERE id=100000")

┌───────┬───────┬───────┬─────────┬─────────┬───┬─────────┬─────────┬────────┬────────┬─────────┬─────────┐
│ year  │  cc   │  mc   │  case   │   cn    │ … │   ocn   │ reason  │  ilon  │  ilat  │   icn   │   id    │
│ int64 │ int64 │ int64 │ varchar │ varchar │   │ varchar │ varchar │ double │ double │ varchar │  int64  │
├───────┼───────┼───────┼─────────┼─────────┼───┼─────────┼─────────┼────────┼────────┼─────────┼─────────┤
│  2004 │     2 │    63 │ 04-077  │ Bergen  │ … │ NULL    │ No MP   │   NULL │   NULL │ NULL    │ 1000000 │
├───────┴───────┴───────┴─────────┴─────────┴───┴─────────┴─────────┴────────┴────────┴─────────┴─────────┤
│ 1 rows                                                                            57 columns (11 shown) │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [37]:
dsql("DESCRIBE crashes")

┌───────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│        column_name        │ column_type │  null   │   key   │ default │ extra │
│          varchar          │   varchar   │ varchar │ varchar │ varchar │ int32 │
├───────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ year                      │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ cc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ mc                        │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ case                      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ cn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ mn                        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdc                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ pdn                       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ station       

In [40]:
%%time
dsql("""
SELECT date_part('year', dt) as year, icn as county, severity, count(*) as num
FROM crashes
WHERE icn IS NOT NULL
GROUP BY (date_part('year', dt), icn, severity)
ORDER BY (date_part('year', dt), icn, severity)
""")

CPU times: user 771 µs, sys: 26 µs, total: 797 µs
Wall time: 809 µs


┌───────┬────────────┬──────────┬───────┐
│ year  │   county   │ severity │  num  │
│ int64 │  varchar   │ varchar  │ int64 │
├───────┼────────────┼──────────┼───────┤
│  2001 │ Atlantic   │ f        │    49 │
│  2001 │ Atlantic   │ i        │  1689 │
│  2001 │ Atlantic   │ p        │  2860 │
│  2001 │ Bergen     │ f        │    16 │
│  2001 │ Bergen     │ i        │  3181 │
│  2001 │ Bergen     │ p        │  7189 │
│  2001 │ Burlington │ f        │    54 │
│  2001 │ Burlington │ i        │  1829 │
│  2001 │ Burlington │ p        │  4050 │
│  2001 │ Camden     │ f        │    33 │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│    ·  │   ·        │ ·        │     · │
│  2021 │ Somerset   │ p        │  5506 │
│  2021 │ Sussex     │ f        │     9 │
│  2021 │ Sussex     │ i        │   423 │
│  2021 │ Sussex     │ p        │  1591 │
│  2021 │ Union      │ f        │    28 │
│  2021 │ Union      │ i        │  3698 │
│  2021 │ Union      │ p        │ 

In [38]:
dsql("EXPLAIN SELECT year, count(*) as num FROM crashes GROUP BY year")

┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ physical_plan │ ┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│__in…  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [39]:
dsql("EXPLAIN ANALYZE SELECT * FROM crashes WHERE id=1000000")

┌───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  explain_key  │                                            explain_value                                             │
│    varchar    │                                               varchar                                                │
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ analyzed_plan │ ┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Prof…  │
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────────┘